# Decrypt & Extract
# What does this do?

## Run global sample accounting script
- Get list of encrypted sample archives (tar.pgp)
- For each archive, check whether a corresponding directory exists
- If not, use dsub to launch decryption job


# Files

* **global-dir-accountant.sh**: Get a list of encrypted tar.pgp archives. 
For each archive, check if there is a corresponding sample directory exists. 
If no corresponding directory found, launch a dsub job to decrypt & extract that tar.pgp archive.
* **sample-dir-accountant.sh**: Same functions as global-dir-accountant.sh except user specifies a single sample to investigate.
* **make-integrity-tsv-from-file.sh**: Create a dsub TSV input file to perform integrity checks on a provided list of input samples.
* **call-integrity-dsub.sh**: Launch dsub jobs to perform integrity checks on all samples listed in TSV input file.
* **list-failed-samples.sh**: Get unique list of samples that included files that were either missing of failed the sha1 check

## 0. Check that environment variables have been loaded correctly

Environment variables are imported from the mvp-profile.sh file. If this echo command does not return anything, try sourcing it from the command console. If any of the values are incorrect, change them in mvp-profile.sh, save it, and source it again.

In [ ]:
# source mvp_profile.sh
echo "Date stamp: ${date_stamp}"
echo "Home directory: ${mvp_hub}"
echo "Project: ${mvp_project}"
echo "Bucket: ${mvp_bucket}"
echo "Zone: ${mvp_zone}"

# Decryption specific variables
echo "Max dsub jobs: ${dsub_max_jobs}"
echo "Encrypted archives path: ${mvp_tar_pgp_path}"
echo "Decyrpted & extracted sample directories path: ${mvp_samples_path}"
echo "Decryption passphrase file path: ${mvp_decrypt_pass}"
echo "Decryption pair.asc file: ${mvp_decrypt_ascpair}"

## 1. Run accounting script to decrypt & extract samples

In [ ]:
${mvp_hub}/decrypt/scripts/global-dir-accountant.sh -d 200GB